# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = "AAPL"
api_url = f"https://api.iex.cloud/v1/data/CORE/QUOTE/{symbol}?token={IEX_CLOUD_API_TOKEN}"
data = requests.get(api_url).json()
data


[{'avgTotalVolume': 64392222,
  'calculationPrice': 'tops',
  'change': 0.3,
  'changePercent': 0.00169,
  'close': None,
  'closeSource': 'official',
  'closeTime': None,
  'companyName': 'Apple Inc',
  'currency': 'USD',
  'delayedPrice': None,
  'delayedPriceTime': None,
  'extendedChange': None,
  'extendedChangePercent': None,
  'extendedPrice': None,
  'extendedPriceTime': None,
  'high': None,
  'highSource': 'IEX real time price',
  'highTime': 1695131111428,
  'iexAskPrice': 178.31,
  'iexAskSize': 100,
  'iexBidPrice': 178.27,
  'iexBidSize': 100,
  'iexClose': 178.27,
  'iexCloseTime': 1695131389178,
  'iexLastUpdated': 1695131389178,
  'iexMarketPercent': 0.019807393770441106,
  'iexOpen': 177.52,
  'iexOpenTime': 1695130200243,
  'iexRealtimePrice': 178.27,
  'iexRealtimeSize': 125,
  'iexVolume': 141934,
  'lastTradeTime': 1695131389178,
  'latestPrice': 178.27,
  'latestSource': 'IEX real time price',
  'latestTime': '9:49:49 AM',
  'latestUpdate': 1695131389178,
  'late

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data[0]['iexRealtimePrice']
market_cap = data[0]['marketCap']
print(f"Price: {price}, Market Capitalization: {market_cap}")

Price: 178.27, Market Capitalization: 2787114538640


## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = ["Ticker", "Stock Price", "Market Capitalization", "Number of Shares to Buy"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [9]:
final_dataframe.append(
    pd.Series(
    [
        symbol,
        price,
        market_cap,
        "N/A"
    ],
    index = my_columns
    ), 
    ignore_index=True
)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,179.94,2804155851520,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [7]:
final_dataframe = pd.DataFrame(columns = my_columns)
for stock in stocks["Ticker"]:
    api_url = f"https://api.iex.cloud/v1/data/core/quote/{stock}?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
    pd.Series(
    [
        stock,
        data[0]['iexRealtimePrice'],
        data[0]['marketCap'],
        "N/A"
    ],
    index = my_columns
    ), 
    ignore_index=True
)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,112.79,33000918522,N/A
1,AAL,13.13,8578648785,N/A
2,AAP,58.32,3468305622,N/A
3,AAPL,178.22,2786332827040,N/A
4,ABBV,154.355,272443780291,N/A
...,...,...,...,...
500,YUM,126.53,35455133385,N/A
501,ZBH,122.27,25550044664,N/A
502,ZBRA,251.61,12917245766,N/A
503,ZION,36.39,5391001390,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [8]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [9]:
symbol_groups = list(chunks(stocks["Ticker"], 2000))
#print(symbol_groups)
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(",".join(symbol_groups[i]))
#    print(symbol_strings[i])
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f"https://api.iex.cloud/v1/data/core/quote/{symbol_string}?token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_api_call_url).json() 
    for x, symbol in enumerate(symbol_string.split(",")):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[x]['latestPrice'],
                data[x]['marketCap'],
                "N/A"
            ],
            index = my_columns),
            ignore_index=True
        )
        
final_dataframe
    

    

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,112.890,33030177249,N/A
1,AAL,13.145,8588449221,N/A
2,AAP,58.340,3469495027,N/A
3,AAPL,177.700,2778203026400,N/A
4,ABBV,154.320,272382003658,N/A
...,...,...,...,...
500,YUM,126.420,35424310144,N/A
501,ZBH,122.270,25550044664,N/A
502,ZBRA,251.610,12917245766,N/A
503,ZION,36.390,5391001390,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [10]:
portfolio_size = input("Enter the value of your portfolio: ")

while True:
    try:
        val = float(portfolio_size)
        break
    except ValueError:
        print("That's not a number \nPlease try again:")
        portfolio_size = input("Enter the value of your portfolio: ")
        continue

Enter the value of your portfolio: 10000000


In [11]:
position_size = val/len(final_dataframe.index)
for i in range(0, len(final_dataframe.index)):
    final_dataframe.loc[i, "Number of Shares to Buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])
final_dataframe
                                                                   

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,112.890,33030177249,175
1,AAL,13.145,8588449221,1506
2,AAP,58.340,3469495027,339
3,AAPL,177.700,2778203026400,111
4,ABBV,154.320,272382003658,128
...,...,...,...,...
500,YUM,126.420,35424310144,156
501,ZBH,122.270,25550044664,161
502,ZBRA,251.610,12917245766,78
503,ZION,36.390,5391001390,544


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWri
ter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [12]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_dataframe.to_excel(writer, "Recommended Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [13]:
background_color = '#0a0a23'
font_color = "#ffffff"

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border':1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border':1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [26]:
#writer.sheets["Recommended Trades"].set_column('A:A', 18, string_format)
#writer.sheets["Recommended Trades"].set_column('B:C', 18, dollar_format)
#writer.sheets["Recommended Trades"].set_column('D:D', 18, integer_format)#
#writer.save()

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [14]:
column_formats = {
    'A':  ['Ticker', string_format],
    'B':  ['Stock Price', dollar_format],
    'C':  ['Market Capitalisation', dollar_format],
    'D':  ['Number of Shares to Buy', integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 18, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


## Saving Our Excel Output

Saving our Excel file is very easy:

In [15]:
writer.save()